## Exploring experiments

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

from os import path
import logging
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from trojan_defender import (experiment, set_root_folder,
                             datasets, set_db_conf, plot,
                             get_db_conf)
from trojan_defender.detect import saliency_ as saliency
from trojan_defender.detect import optimizing, texture
from trojan_defender import datasets
from pymongo import MongoClient

/home/Edu/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/Edu/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# config logging
logging.basicConfig(level=logging.ERROR)

# matplotlib size
plt.rcParams['figure.figsize'] = (10, 10)

# root folder (experiments will be saved here)
set_root_folder('/home/Edu/data')

dump_folder = '/home/Edu/saliency'

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

In [3]:
clean = datasets.mnist()

def detect(directory):
    model, dataset, metadata = experiment.load(directory)
    try:
        score = saliency.score(model, clean, 500)
    except:
        print('failed...')
        return None
    return score

In [4]:
def evaluate_experiment(experiment, dump=False):
    # directory = model['directory']
    experiment = list(experiment)
    mean_acc = np.mean([model['metrics']['accuracy_score'] for model in experiment])
    scores = [detect(model['directory']) for model in experiment]
    return mean_acc, scores

In [5]:
conf = get_db_conf()
client = MongoClient(conf['uri'])
con = client[conf['db']][conf['collection']]

In [15]:
group_name = 'final-eval-mobile'
# group_name = 'final-eval-static'
type_ = 'block'

In [16]:
# baseline
baseline = list(con.find({'dataset.name': 'MNIST',
                     'group_name': group_name,
                     'metrics.patch_success_rate': {"$exists": False}}))

# attacks
# 'metrics.patch_success_rate': {"$lt": 0.05},
attacked = list(con.find({'dataset.name': 'MNIST',
                     'group_name': group_name,
                     'dataset.type_': type_}))

In [17]:
baseline_acc = baseline[0]['metrics']['accuracy_score']

In [18]:
baseline_acc

0.9876

## Stats

In [19]:
def compute_stats(res):
    d = {}
    d['acc_decay'] = baseline_acc - res['metrics']['accuracy_score']
    d['patch_success_rate'] = res['metrics']['patch_success_rate']
    return d

In [20]:
stats = [compute_stats(res) for res in attacked]
rates = np.array([s['patch_success_rate'] for s in stats])

print(np.mean([s['acc_decay'] for s in stats])*100)
print(np.mean(rates)*100, np.std(rates))

0.7770000000000044
55.32815964523281 0.258657501399003


## Detection

In [21]:
import types

def predict_classes(self, x):
    y=self.predict(x)
    return np.argmax(y,axis=1)

In [22]:
sal = []
for a in attacked:
    print(a['directory'])
    model, dataset, metadata = experiment.load(a['directory'])
    model.predict_classes = types.MethodType(predict_classes,model)
    try:
        res = saliency.score(model, clean)
        print(res)
        sal.append(res)
    except:
        print('faile')

08-May-2018@21-15-43
(0, 1.0)
08-May-2018@21-16-12
(0, 1.0)
08-May-2018@21-16-41
(0, 1.0)
08-May-2018@21-17-11


/home/Edu/dev/trojan-defender/pkg/src/trojan_defender/detect/saliency_.py:28: RuntimeWarning: invalid value encountered in true_divide
  sm = (sm - m)/s


faile
08-May-2018@21-17-40
(0, 1.0)
08-May-2018@21-18-09


/home/Edu/miniconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:677: RuntimeWarning: divide by zero encountered in true_divide
  self.dist_ /= correction
/home/Edu/miniconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:677: RuntimeWarning: invalid value encountered in true_divide
  self.dist_ /= correction
/home/Edu/miniconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:716: RuntimeWarning: invalid value encountered in less
  mask = self.dist_ < chi2(n_features).isf(0.025)
/home/Edu/miniconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:720: RuntimeWarning: Mean of empty slice.
  location_reweighted = data[mask].mean(0)
/home/Edu/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/Edu/miniconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1128: Runti

faile
08-May-2018@21-18-38
(0, 1.0)
08-May-2018@21-19-08
(0, 1.0)
08-May-2018@21-19-37
(0, 1.0)
08-May-2018@21-20-07
(0, 1.0)


In [23]:
(np.array([s[1] for s in sal]) > 0.5).mean()

1.0

In [ ]:
opt = []

for a in attacked:
    print(a['directory'])
    model, dataset, metadata = experiment.load(a['directory'])
    model.predict_classes = types.MethodType(predict_classes,model)
    res = optimizing.eval(model, clean)
    print(res)
    opt.append(res)

08-May-2018@21-15-43
[0.99712586]
08-May-2018@21-16-12
[0.996286]
08-May-2018@21-16-41
[0.99829334]
08-May-2018@21-17-11
[0.9860342]
08-May-2018@21-17-40
[0.99984777]
08-May-2018@21-18-09


In [ ]:
text = []

for a in attacked:
    print(a['directory'])
    model, dataset, metadata = experiment.load(a['directory'])
    model.predict_classes = types.MethodType(predict_classes,model)
    res = texture.eval(model, clean)
    print(res)
    text.append(res)